In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Load data
df = pd.read_csv("BITCOIN_DATA.csv")
df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')
df = df.sort_values(by='Ngày')
df['Lần cuối'] = df['Lần cuối'].str.replace(',', '').astype(float)
df.set_index('Ngày', inplace=True)
df = df[['Lần cuối']]

In [ ]:
# ADF Test
result = adfuller(df['Lần cuối'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
# Differencing
df_diff = df.diff().dropna()

# Plot differenced series
plt.figure(figsize=(12, 5))
plt.plot(df_diff, label='Sai phân bậc 1')
plt.title('Chuỗi sai phân bậc 1 của giá Bitcoin')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# ACF and PACF
fig, ax = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(df_diff, ax=ax[0])
plot_pacf(df_diff, ax=ax[1])
plt.tight_layout()
plt.show()

In [ ]:
# Fit ARIMA model manually with selected order (2, 2, 3)
model = ARIMA(df['Lần cuối'], order=(2, 2, 3))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# Forecast
forecast_steps = 365  # 1 year forecast
forecast = model_fit.forecast(steps=forecast_steps)

# Plot forecast
plt.figure(figsize=(12, 5))
plt.plot(df.index[-100:], df['Lần cuối'].iloc[-100:], label='Thực tế')
plt.plot(pd.date_range(df.index[-1], periods=forecast_steps + 1, freq='D')[1:], forecast, label='Dự báo')
plt.title('Dự báo giá Bitcoin cho năm 2025')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# In-sample prediction and evaluation
pred = model_fit.predict(start=0, end=len(df) - 1)
rmse = np.sqrt(mean_squared_error(df['Lần cuối'][2 + 3:], pred[2 + 3:]))  # skip warm-up
mae = mean_absolute_error(df['Lần cuối'][2 + 3:], pred[2 + 3:])
mape = mean_absolute_percentage_error(df['Lần cuối'][2 + 3:], pred[2 + 3:])
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}')